In [1]:
# Examples related to data processing, last updated 11/2/2021, Kluster 0.8.2
# uses the multibeam file hstb_kluster/test_data/0009_20170523_181119_FA2806.all
# Written by Eric Younkin

# data processing can be done in one of three ways:
#  1. piece-wise lower level approach = use convert_multibeam, process_multibeam
#  2. merged lower level approach = use perform_all_processing
#  3. high level automated processing = use the intel process/service

# --- SKIP TO 3 FOR THE RECOMMENDED PROCESSING ROUTINE --- #

In [2]:
from HSTB.kluster.fqpr_convenience import convert_multibeam, process_multibeam, import_processed_navigation, \
    perform_all_processing, reload_data
from HSTB.kluster.fqpr_intelligence import intel_process, intel_process_service

ImportError: cannot import name 'import_processed_navigation' from 'HSTB.kluster.fqpr_convenience' (C:\PydroXL_19\NOAA\site-packages\Python38\HSTB\kluster\fqpr_convenience.py)

In [ ]:
#####################################
# 1. piece-wise lower level approach
#####################################

# conversion will generate data in the Kluster zarr/xarray format for you to use
# can either be a list of multibeam files
# fq = convert_multibeam([r"C:\data_dir\0009_20170523_181119_FA2806.all",])
# a path to a single file
# fq = convert_multibeam(r"C:\data_dir\\0009_20170523_181119_FA2806.all")
# or a path to a directory of files
# fq = convert_multibeam(r"C:\data_dir")

In [ ]:
fq = convert_multibeam(r"C:\data_dir\0009_20170523_181119_FA2806.all")

In [ ]:
# look at the summary of the converted data
fq

In [ ]:
# the converted ping datasets are in fq.multibeam.raw_ping.  There is one dataset for each sonar head.  These datasets
#  have (time, beam) dimensions, where each time value is a new ping.

In [ ]:
# number of heads
len(fq.multibeam.raw_ping)

In [ ]:
# dataset for the first (and only) head
fq.multibeam.raw_ping[0]

In [ ]:
# number of pings
fq.multibeam.raw_ping[0].time.size

In [ ]:
# number of beams
fq.multibeam.raw_ping[0].beam.size

In [ ]:
# fq.multibeam.raw_ping[0] is an xarray dataset, so we can access data using the dataset api
fq.multibeam.raw_ping[0].beampointingangle

In [ ]:
# you might have some data that is not in the multibeam file, for instance:
# loading from a post processed Applanix SBET file
fq = import_processed_navigation(fq, [r'C:\data_dir\sbet.out'], [r'C:\data_dir\smrmsg.out'], [r'C:\data_dir\export_log.txt'])

In [ ]:
# now you can process the converted/imported data.  The defaults will do a full processing run on all data, but you can
#  specify a few things if you like
# the default run
fq = process_multibeam(fq)

In [ ]:
# include another sound velocity file
fq = process_multibeam(fq, add_cast_files=r'C:\data_dir\mysvpfile.svp')

In [ ]:
# specify coordinate system and vertical reference
fq = process_multibeam(fq, coord_system='WGS84', vert_ref='ellipse')

In [ ]:
# reload the data later on
fq = reload_data(r"C:\data_dir\converted")

In [ ]:
#####################################
# 2. merged lower level approach
#####################################

# all the above can be combined into the perform_all_processing command for ease of use
fq = perform_all_processing(r"C:\data_dir\0009_20170523_181119_FA2806.all", navfiles=[r'C:\data_dir\sbet.out'],
                            errorfiles=[r'C:\data_dir\smrmsg.out'], logfiles=[r'C:\data_dir\export_log.txt'],
                            add_cast_files=r'C:\data_dir\mysvpfile.svp', coord_system='WGS84', vert_ref='ellipse')

In [ ]:
# reload the data later on
fq = reload_data(r"C:\data_dir\converted")

In [ ]:
########################################################################
# 3. high level automated processing = use the intel process/service
########################################################################

# when you drag in new multibeam data into Kluster, it generates a new conversion action and organizes data into:
#  sonarmodelnumber_serialnumber_dateofsurvey
# which it gets from the multibeam data itself.  This is the Kluster Intelligence module that will basically perform
#  the convert_multibeam and process_multibeam actions for you, putting data in the right place and doing only those
#  steps that are required.  For this reason it is recommended that you use the intelligence module rather than
#  the core processing routines described in 1 and 2.  Learn more here: https://kluster.readthedocs.io/en/latest/indepth/intel.html

# the intel process command will perform just like if you were to drag in new files in Kluster.  You just provide all the
#  files that you want to add, and Kluster Intelligence determines the type of file, how to add it and what processing
#  steps to take.  Those steps are all performed and you get the Fqpr object back


In [ ]:
# just like all the others, either provide a list of files...
_, fq = intel_process([r"C:\data_dir\0009_20170523_181119_FA2806.all",
                       r'C:\data_dir\sbet.out', r'C:\data_dir\smrmsg.out',
                       r'C:\data_dir\export_log.txt', r'C:\data_dir\mysvpfile.svp'],
                      coord_system='WGS84', vert_ref='ellipse')

In [ ]:
# a single file or a path to a directory full of files:
_, fq = intel_process(r"C:\data_dir")

In [ ]:
# it will behave much like step 2, you probably won't notice a difference during the processing
# what you should notice is that the output directory is now a folder with the sonarmodelnumber_serialnumber_dateofsurvey of the file
# - note: fq is a list of the converted fqpr objects, since data is organized by modelnumber,etc., you might end up with multiple containers
fq[0].output_folder

In [ ]:
# the last thing to mention is the intel_process_service, which combines intel_process with folder monitoring.  The service
#  will monitor a folder and add/process any files that you add to that directory (or are already in there)
_, fq = intel_process_service(r"C:\data_dir")
# this will lock up the console until you force it to quit

In [ ]:
# reload the data later on
fq = reload_data(r"C:\data_dir\em2040_40111_05_23_2017")